# TP: Essaims de nano-satellites -- *Nanosatellite swarms*

Ce TP a pour objectif de vous familiariser davantage avec les topologies d'essaims de nano-satellites en exploitant un jeu de données. Vous verrez notamment qu'on peut extraire énormément d'informations à partir d'un jeu de données simple (ici, positions des satellites par pas de temps). Le TP se déroule en 3 parties :  

1. Chargement des données : découverte du jeu de données et formattage pour la suite
2. Visualisation de la topologie
3. Partage de charge : division en sous-réseaux

Nous allons d'abord importer les librairies nécessaires, dont le module de simulation `swarm_sim` disponible dans le dossier `\tp`.  

***

*The objective of this TP is to become familiar with nanosatellite swarms topologies extracted from a given dataset. You'll notice in particular that one can extract a lot of information from a rather simple dataset (here, temporal satellite positions). This TP is divided in 3 parts:*  

1.  *Data loading: dataset exploration and initialization*
2.  *Visualization of the topologies*
3.  *Data load balancing: division into sub-networks*  

*We will first import and the necessary Python libraries, namely the simulation module* `swarm_sim` *that you can find under the* `\tp` *directory.*  

*Make sure you choose the* `.venv` *environment for execution.*

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from swarm_sim import *

## 1. Chargement des données -- *Data loading*

Les données sont stockées dans le dossier `\data\swarm-50-sats-scenario` du repository Git, et sont réparties en 50 traces de nano-satellites échantillonnées sur 24 heures.  

Ouvrez une trace en exécutant la cellule suivante, et rappelez les champs ainsi que les unités utilisées (n'hésitez pas à revoir le cours, slide 20).

***

*The dataset is available under the* `\data\swarm-50-sats-scenario` *folder of the Git repository. It consists of 50 separate nanosatellite tracks sampled over 24 hours.*  

*Open one track by executing the following cell, and pay attention to the fields and units (slide 35).*

In [ ]:
PATH = '..\\data\\swarm-50-sats-scenario\\coords_v1_if_LLO-' #You might need to adapt, depending on the OS
NB_NODES = 50
DURATION = 8641 # Number of data samples, not time!
REVOLUTION = 1800 # Number of data rows to complete a revolution around the Moon
CONNECTION_RANGE = 30 # km

row_metadata_end = 6
row_data_start = 7

sat_id = 0
df_metadata = pd.read_csv(PATH+str(sat_id)+'.csv', skiprows = lambda x: x>row_metadata_end)
df_data = pd.read_csv(PATH+str(sat_id)+'.csv', skiprows= lambda x: x<row_data_start, header=0)
        
print(df_metadata)
df_data.head()

Chargez la totalité des données dans ce notebook. Nous allons créer un dictionnaire contenant ces données suivant ce format : 

`satellites[id] = track`

Modifiez la variable `PATH` si nécessaire.

NB : le chargement peut être assez long (plusieurs minutes en fonction de la machine). Vous pouvez visualiser la progression du chargement avec la fonction `tqdm()`.  

***

*You can now load the whole dataset into a Python dictionary formatted as follows:*  

`satellites[id] = track`  

*Modify the* `PATH` *variable if necessary.*  

*NB: the loading can take some time (sometimes a couple minutes, depending on the machine). You can visualize the progression of the loading with the* `tqdm()` *function.*

In [ ]:
metadata = {}
satellites = {}

with tqdm(total=NB_NODES, desc='Extracting data') as pbar:
    for i in range(NB_NODES):
        df_metadata = pd.read_csv(PATH+str(i)+'.csv', skiprows = lambda x: x>row_metadata_end)
        metadata[i] = df_metadata
        df_data = pd.read_csv(PATH+str(i)+'.csv', skiprows= lambda x: x<row_data_start, header=0)
        satellites[i] = df_data
        pbar.update(1)
        
satellites[0].head()

Afin de faciliter la suite du traitement, nous allons créer un objet `Swarm` tel que défini dans le module `swarm_sim` par pas de temps et les stocker dans un dictionnaire tel que :

`swarm_data[timestamp] = Swarm`

N'hésitez pas à lire la doc pour le formattage en objet `Swarm`.  

***

*In order to simplify the following data analysis, we will create* `Swarm` *objects as defined in the* `swarm_sim` *module for each timestamp, and stock them in a dictionary such as:*  

`swarm_data[timestamp] = Swarm`

*Don't hesitate to take a look at the documentation to understand the formatting process.*

In [ ]:
help(Swarm.__init__)
help(Node.__init__)

In [ ]:
swarm_data = {}

with tqdm(total=DURATION, desc='Converting to Swarm') as pbar:
    for t in range(DURATION):
        swarm_data[t] = Swarm(connection_range=CONNECTION_RANGE,
                    nodes = [Node(id, sat['xF[km]'].iloc[t], sat['yF[km]'].iloc[t], sat['zF[km]'].iloc[t]) for id,sat in satellites.items()]
                    )
        pbar.update(1)

Affichez le contenu du `swarm_data` à l'instant `0`, ainsi que la description d'un noeud de votre choix. Assurez-vous de bien comprendre tous les champs affichés.  

***

*Print the content of* `swarm_data` *at timestamp* `0`*, as well as the description of a node of your choosing. Make sure you understand all the fields.*

In [ ]:
swarm = swarm_data[0]

# To do

## 2. Visualisation de la topologie -- *Topology visualization*  

Le module `swarm_sim` permet notamment de créer des graphiques 3D représentant les positions des satellites à un instant donné, avec si besoin les ISL (liens inter-satellites) existants. Il s'agit des fonctions `plot_nodes()` et `plot_edges()`.  

Affichez la topologie de l'essaim à un pas de temps donné, puis à un autre.  

Qu'observez-vous au niveau de la topologie ?  

***

*The* `swarm_sim` *module allows you to create 3D graphs representing the positions of the nanosatellites at a given time, and if needed, the existing ISL (inter-satellite links). To do so, you can use the functions* `plot_nodes()` *and* `plot_edges()`.  

*Display the swarm topology at a given time (you choose), then at another further away.*  

*What do you observe on the topologies?*

In [ ]:
swarm_data[0].plot_nodes()
swarm_data[0].plot_edges()

Vous avez sûrement remarqué que la fonction `plot_edges()` ne fonctionne pas, ou du moins elle n'affiche aucune connexion dans l'essaim. Pourquoi ?  

***

*You probably noticed that* `plot_edges()` *doesn't actually return any edge. Why so?*

In [ ]:
""" 
Réponse -- answer
"""

Etablissez les connexions entre noeuds voisins grâce à la fonction `neighbor_matrix()` (regardez la doc pour comprendre comment elle fonctionne), puis affichez la topologie de l'essaim grâce à la fonction `plot_edges()`.  

***

*Establish the connections between neighbor nodes with the function* `neighbor_matrix()` *(take a look at the doc to understand how it works), then display the swarm topology with* `plot_edges()`*.*

In [ ]:
help(Swarm.neighbor_matrix)

In [ ]:
with tqdm(total=DURATION, desc='Neighbor matrix') as pbar:
    for t in range(DURATION):
        neighbor_matrix = swarm_data[t].neighbor_matrix()
        pbar.update(1)

In [ ]:
swarm_data[0].plot_edges()

## 3.  Partage de charge -- *Load balancing*

Nous allons maintenant nous intéresser à la gestion de la redondance et du recouvrement dans le réseau (slides 29+ du cours).

Pour rappel, lors d'une mission d'interférométrie, chaque nanosatellite va collecter près de 5 Gb de données brutes de l'espace. Afin de créer l'image globale collectée par l'essaim, toutes les données collectées doivent être échangées entre l'intégralité des satellites. Sans politique de limitation de recouvrement, le réseau va saturer très vite.

Afin de limiter ce recouvrement, une solution est de diviser le réseau en plusieurs sous-réseaux grâce à des algorithmes de **division de graphe**. En effet, les sous-réseaux obtenus vont faire office de "noeuds" dans le graphe simplifié, et on va ainsi limiter la quantité de données échangées simultanément. Nous allons analyser les performances de trois algorithmes: **Random Node Division** (`RND`), **Multiple Independent Random Walk** (`MIRW`) et **Forest Fire Division** (`FFD`).

Cherchez les fonctions correspondantes à ces algorithmes dans la doc, et assurez-vous de bien comprendre leur principe de fonctionnement. Quelles sont les différences majeures dans leur implémentation ?  

***

*We will now focus on the redundancy management and network overload control (slides 27+).*  

*Reminder: during an interferometry mission, each nanosatellite collects approx. 5 Gb of space observation raw data. IN order to create the global image collected by the swarm, all data packets need to be shared among all the satellites of the swarm. Without overload control, the network will experience strong congestion.*  

*In order to manage the data overload, one solution is to divide the network into sub-networks by using* **Graph Division** *algorithms. The resulting sub-networks can indeed act as big aggregated nodes in a simplified version of the graph, and thus limit the amount of data to transmit. We will analyze the performance of 3 division algorithms:* **Random Node Division** (`RND`), **Multiple Independent Random Walk** (`MIRW`) *and* **Forest Fire Division** (`FFD`).  

*Look for these algorithms in the documentation and take a look at how they operate. What are the main differences in their respective implementations?*

In [ ]:
help(Swarm.RND) # Test with MIRW and FFD

In [ ]:
""" 
Réponse -- answer
"""

Prenez un essaim à un instant `T` et appliquez-lui d'abord l'algorithme `RND`. Paramétrez-le de sorte à obtenir `5` groupes.  

***

*Take a swarm at a given time* `T` *and start by applying the* `RND` *algorithm. Configure it to obtain a division into* `5` *groups.*

In [ ]:
T = 0
NB_GROUPS = 1
swarm = swarm_data[T]

In [ ]:
#It's a good practice to reset the swarm division to its default value before starting (-1)
swarm.reset_groups() 
swarms_rnd = swarm.RND(n=NB_GROUPS)

for i,sw in swarms_rnd.items():
    print(sw)
    print([n.id for n in sw.nodes])

Faites maintenant la même chose mais avec l'algorithme `MIRW` afin d'obtenir une autre répartition.  

***

*Now do the same with the* `MIRW` *algorithm in order to obtain a different distribution.*

In [ ]:
help(Swarm.MIRW)

In [ ]:
swarm.reset_groups()
swarms_mirw = swarm.MIRW(n=NB_GROUPS) 

for i,sw in swarms_mirw.items():
    print(sw)
    print([n.id for n in sw.nodes])

Enfin, obtenez une troisième répartition grâce à l'algorithme `FFD`.  

***

*Finally, get one last repartition with* `FFD`.

In [ ]:
help(Swarm.FFD)

In [ ]:
swarm.reset_groups()
swarms_ffd = swarm.FFD(n=NB_GROUPS)

for i,sw in swarms_ffd.items():
    print(sw)
    print([n.id for n in sw.nodes])

A présent, nous allons comparer ces algorithmes en fonction de la répartition de la **taille des groupes** obtenus, l'idéal étant d'obtenir la répartition la plus homogène possible.

Exécutez les cellules suivantes afin de générer les graphiques correspondant aux répartitions des trois algorithmes. Adaptez le nom des variables si besoin.  

***

*We will now compare the performance of these 3 algorithms on the distribution of the* **group size**. *The objective is to obtain the fairest distribution of group sizes.*  

*Execute the following cells to generate figures corresponding to the group size distribution of each algorithm. Adapt the variable names if necessary.*

In [ ]:
distrib_rnd = [len(sw.nodes) for sw in swarms_rnd.values()] # Group size distribution
distrib_mirw = [len(sw.nodes) for sw in swarms_mirw.values()]
distrib_ffd = [len(sw.nodes) for sw in swarms_ffd.values()]

values = [] # Variable used to compare all distributions on the same scale
values.extend(distrib_rnd)
values.extend(distrib_mirw)
values.extend(distrib_ffd)

In [ ]:
labels = sorted(set(values))
x_pos = np.arange(min(labels), max(labels)+1)
data_rnd, data_mirw, data_ffd = [], [], []
for k in x_pos:
       a,b,c = 0,0,0
       if k in distrib_rnd:
            a = len([e for e in distrib_rnd if e==k])
       data_rnd.append(a)
       if k in distrib_mirw:
            b = len([e for e in distrib_mirw if e==k])
       data_mirw.append(b)
       if k in distrib_ffd:
            c = len([e for e in distrib_ffd if e==k])
       data_ffd.append(c)

# Figure generation
fig, axes = plt.subplots(nrows=3, figsize=(12,12))
ax = axes[0] # Histogramme RND
ax.bar(x_pos, data_rnd,
       align='center',
       alpha=0.5)
ax.set_ylabel('# Occurrences')
ax.set_xticks(x_pos)
ax.set_xticklabels(x_pos)
ax.set_ylim(0, NB_GROUPS)
ax.set_title('RND distribution')
ax.yaxis.grid(True)

ax = axes[1] # Histogramme MIRW 
ax.bar(x_pos, data_mirw,
       align='center',
       alpha=0.5)
ax.set_ylabel('# Occurrences')
ax.set_xticks(x_pos)
ax.set_xticklabels(x_pos)
ax.set_ylim(0, NB_GROUPS)
ax.set_title('MIRW distribution')
ax.yaxis.grid(True)

ax = axes[2] # Histogramme FFD 
ax.bar(x_pos, data_ffd,
       align='center',
       alpha=0.5)
ax.set_xlabel('# Nodes in group')
ax.set_ylabel('# Occurrences')
ax.set_xticks(x_pos)
ax.set_xticklabels(x_pos)
ax.set_ylim(0, NB_GROUPS)
ax.set_title('FFD distribution')
ax.yaxis.grid(True)

Ici, nous avons effectué une division à `10%` (5 groupes). Quel algorithme semble le plus adapté dans ce cas ? Le moins adapté ? Pourquoi ?  

***

*Here, we have performed a* `10%`*-division (5 groups). Which algorithm seems to be the best fitted in your opinion? Why so?*

In [ ]:
""" 
Réponse -- answer
"""

Répétez les opérations précédentes, mais cette fois-ci en effectuant une division en **2**, **7** puis **10** groupes.  

NB : pour être rigoureux, il faudrait répéter chaque expérience un grand nombre de fois, car les 3 algorithmes se basent sur de l'aléatoire (d'où le paramètre "seed" dans les fonctions). Si cela est trop long à réaliser, vous pouvez choisir des seeds différentes de vos voisins afin de confronter vos résultats.  

***

*Repeat the previous operation, but now divide the swarm respectively into* **2**, **7** *and* **10** *groups.*  

*NB: because all 3 algorithms are based on random processes (the "seed" parameter), you should repeat each operation at least 30 times to get a rigorous and reliable result. It would probably take too long to do that, so instead you can simply choose different seeds from your classmates, then compare your results.*

In [ ]:
# A faire -- To do

D'après vos résultats et ceux de vos voisins, comment évolue la performance des algorithmes lorsqu'on augmente le nombre de groupes ? Qu'en concluez-vous sur la performance globale de ces algorithmes sur la division d'un essaim de nanosatellites ?  

***

*From your results and those of the class, how do the algorithms perform when you vary the group size? What can you conclude on the overall performance of these algorithms on the graph division of a nanosatellite swarm?*

In [ ]:
""" 
Réponse -- answer
"""

### Pour aller plus loin : estimation de la charge réseau -- Network load estimation

Comme énoncé dans le cours (slides 31-32), la division en groupes a un fort impact sur le nombre de paquets de données à émettre dans l'essaim. La charge du réseau est estimée par la fonction `network_load()` définie ci-dessous, et est égale à la somme des paquets à échanger au sein du groupe (`total_intra`) et avec les autres groupes (`total_inter`).

Cette fonction considère 2 cas :
 * situation d'équité (`fair = True`) : les noeuds sont répartis équitablement au sein des groupes
 * situation de non-équité : le nombre de noeuds varie en fonction du groupe.

 Compléter cette fonction afin de considérer le 2e cas.  

 ***

*As stated during the lecture (slide 29), graph division has an important impact on the number of data packets to transmit within the swarm. The network load is estimated by the function* `network_load()` *defined below, and is equal to the sum of packets to transmit within the group* (`total_intra`) *and with the other groups* (`total_inter`).  

*This function considers 2 cases:*  
* *the division is fair* (`fair = True`): *the nodes are fairly distributed among the groups*
* *the division is not fair: the number of nodes varies with the groups.*

*Complete this function in order to consider the 2nd case.*

In [ ]:
def network_load(swarm, nb_groups, fair=True):
    total_nodes = len(swarm.nodes)
    
    if fair:    
        nodes_per_group = total_nodes/nb_groups
        total_intra = nb_groups * nodes_per_group*(nodes_per_group-1)
    else:
        # To complete (remove 'pass' when you're done!)
        pass
    
    total_inter = nb_groups*(nb_groups-1)  # Don't change
    net_load = total_intra+total_inter
    print('Network load:', net_load, 'packets.')
    return net_load

Testez cette fonction sur la topologie (divisée) de votre choix : calculez le nombre de paquets à transmettre relatif à la division de graphe en situation réelle et en situation d'équité.

In [ ]:
# A faire

On peut assez vite comprendre pourquoi on cherche à effectuer une division équitable !

***

*You can easily see why we want a fair division!*  
*Congrats, you've made it this far.*